In [39]:
%load_ext autoreload
%autoreload 2
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import logging
logging.basicConfig(level=logging.INFO)

import pyrosetta
import pyrosetta.distributed.io as io
import pyrosetta.distributed.packed_pose as packed_pose
import pyrosetta.distributed.tasks.rosetta_scripts as rosetta_scripts
import pyrosetta.distributed.tasks.score as score

import pandas as pd
import pandas

import numpy as np
from glob import glob
import os

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
import truncator

In [41]:
print(pyrosetta.version())

PyRosetta-4 2019 [Rosetta devel 2019.01+HEAD.dbc838b6ae620b1293476b1bd4366ffc2facc5b5 2019-01-03T10:31:13] retrieved from: ssh://git@github.com/RosettaCommons/main
(C) Copyright Rosetta Commons Member Institutions.
Created in JHU by Sergey Lyskov and PyRosetta Team.



In [42]:
import sys; sys.path.insert(0, "/home/ajasja/projects/Stable_Heterodimers")
import utils as u

In [43]:
ON_CLUSTER=False

In [44]:
flags = """
-beta \
-renumber_pdb 0 \
-overwrite 1 \
-out:pdb \
-holes:dalphaball truncator/scripts/DAlphaBall.gcc \
-multi_cool_annealer 10 \
"""

In [45]:
if not ON_CLUSTER:
    pyrosetta.init(flags)
else:
    cluster = u.init_env(flags, 10)

INFO:pyrosetta.rosetta:Found rosetta database at: /software/conda/envs/pyrosetta/lib/python3.7/site-packages/pyrosetta/database; using it....
INFO:pyrosetta.rosetta:PyRosetta-4 2019 [Rosetta devel 2019.01+HEAD.dbc838b6ae620b1293476b1bd4366ffc2facc5b5 2019-01-03T10:31:13] retrieved from: ssh://git@github.com/RosettaCommons/main
(C) Copyright Rosetta Commons Member Institutions.
Created in JHU by Sergey Lyskov and PyRosetta Team.

INFO:rosetta:core.init: Checking for fconfig files in pwd and ./rosetta/flags
INFO:rosetta:core.init.score_function_corrections: [ WARNING ] Flag -beta_nov16 is set but -weights are also specified.  Not changing input weights file!
INFO:rosetta:core.init: Rosetta version: 2019.01+HEAD.dbc838b6ae6 dbc838b6ae620b1293476b1bd4366ffc2facc5b5 ssh://git@github.com/RosettaCommons/main 2019-01-03T10:31:13
INFO:rosetta:core.init: command: PyRosetta -beta -renumber_pdb 0 -overwrite 1 -out:pdb -holes:dalphaball truncator/scripts/DAlphaBall.gcc -multi_cool_annealer 10 -data

In [79]:
def lines_grep(pattern, lines):
    import re
    reg = re.compile(pattern)
    return [l for l in lines if reg.match(l)]

def print_sele(scores, name, color=None, remove_chains=False):
    sele = scores[name+"_pymol_selection"]
    sele=sele.replace("rosetta_sele", name)
    if remove_chains:
        sele=sele.replace("chain A and", "").replace("chain B and", "")
    print(sele)
    if not color is None:
        print(f"color {color}, {name}")
    

In [84]:
%%time
#Pyrosetta unsupported BuriedUnsatHbonds2 InterfaceHoles CavityVolume
#pack_protocol = truncator.read_file("truncator/xml/05_specific_interface_markers.xml")
pack_protocol = truncator.read_file("truncator/xml/31_fix_surf.xml")
#print(lines_grep(".*Neighborhood.*", pack_protocol.split("\n")))
task_design = rosetta_scripts.SingleoutputRosettaScriptsTask(pack_protocol)
logging.basicConfig(level=logging.WARN)
task_design.setup()

INFO:rosetta:protocols.rosetta_scripts.RosettaScriptsParser: Validating input script...
INFO:rosetta:protocols.rosetta_scripts.RosettaScriptsParser: ...done
INFO:rosetta:protocols.rosetta_scripts.RosettaScriptsParser: Parsed script:
<ROSETTASCRIPTS>
	<SCOREFXNS>
		<ScoreFunction name="bbmin_hard_cart" weights="beta_cart">
			<Reweight scoretype="coordinate_constraint" weight="2.0"/>
			<Reweight scoretype="cart_bonded" weight="0.5"/>
		</ScoreFunction>
		<ScoreFunction name="hard" weights="beta"/>
		<ScoreFunction name="hard_cst" weights="beta_cst"/>
		<ScoreFunction name="hard_cart" weights="beta_cart"/>
		<ScoreFunction name="soft_cst" weights="/home/sboyken/weights/beta_soft_rep_cst.wts">
			<Reweight scoretype="aa_composition" weight="1.0"/>
		</ScoreFunction>
		<ScoreFunction name="hard_cst_cord" weights="beta_cst">
			<Reweight scoretype="coordinate_constraint" weight="0"/>
			<Reweight scoretype="aa_composition" weight="0"/>
			<Reweight scoretype="angle_constraint" weight="0"/>

CPU times: user 425 ms, sys: 68.1 ms, total: 493 ms
Wall time: 572 ms


In [85]:
# Syntax check via setup

raw_input_pose = io.pose_from_file(filename=r"data\ZCON_37_N3_C1_2.pdb".replace("\\","/"))
output_pose = task_design(raw_input_pose)

output_pose.scores

INFO:rosetta:core.import_pose.import_pose: File 'data/ZCON_37_N3_C1_2.pdb' automatically determined to be of type PDB
INFO:rosetta:protocols.rosetta_scripts.RosettaScriptsParser: Validating input script...
INFO:rosetta:protocols.rosetta_scripts.RosettaScriptsParser: ...done
INFO:rosetta:protocols.rosetta_scripts.RosettaScriptsParser: Parsed script:
<ROSETTASCRIPTS>
	<SCOREFXNS>
		<ScoreFunction name="bbmin_hard_cart" weights="beta_cart">
			<Reweight scoretype="coordinate_constraint" weight="2.0"/>
			<Reweight scoretype="cart_bonded" weight="0.5"/>
		</ScoreFunction>
		<ScoreFunction name="hard" weights="beta"/>
		<ScoreFunction name="hard_cst" weights="beta_cst"/>
		<ScoreFunction name="hard_cart" weights="beta_cart"/>
		<ScoreFunction name="soft_cst" weights="/home/sboyken/weights/beta_soft_rep_cst.wts">
			<Reweight scoretype="aa_composition" weight="1.0"/>
		</ScoreFunction>
		<ScoreFunction name="hard_cst_cord" weights="beta_cst">
			<Reweight scoretype="coordinate_constraint" we

{'psi_pred': 0.9831932783126831,
 'fa_atr': -743.5753590413335,
 'fa_rep': 116.77513988877328,
 'fa_sol': 663.6928683437086,
 'fa_intra_atr_xover4': -51.558366468746854,
 'fa_intra_rep_xover4': 30.185345231730935,
 'fa_intra_sol_xover4': 43.21710333529356,
 'lk_ball': 375.6713846885276,
 'lk_ball_iso': 903.155511424895,
 'lk_ball_bridge': 1.9278497346033625,
 'lk_ball_bridge_uncpl': 12.636082331526154,
 'fa_elec': -371.0581733006344,
 'fa_intra_elec': -23.750217692233267,
 'pro_close': 0.0,
 'hbond_sr_bb': -131.23426068055278,
 'hbond_lr_bb': 0.0,
 'hbond_bb_sc': -2.4363384171592926,
 'hbond_sc': -43.67466710860555,
 'dslf_fa13': 0.0,
 'omega': 6.611153259821561,
 'fa_dun_dev': 8.992491434239561,
 'fa_dun_rot': 124.7017873669296,
 'fa_dun_semi': 160.12623193081222,
 'p_aa_pp': -39.03770718043659,
 'hxl_tors': 8.801517200407915,
 'ref': -94.55407000000004,
 'rama_prepro': -16.406240476720058,
 'gen_bonded': 0.0,
 'total_score': -470.6553446826023}

In [86]:
s =output_pose.scores; s

{'psi_pred': 0.9831932783126831,
 'fa_atr': -743.5753590413335,
 'fa_rep': 116.77513988877328,
 'fa_sol': 663.6928683437086,
 'fa_intra_atr_xover4': -51.558366468746854,
 'fa_intra_rep_xover4': 30.185345231730935,
 'fa_intra_sol_xover4': 43.21710333529356,
 'lk_ball': 375.6713846885276,
 'lk_ball_iso': 903.155511424895,
 'lk_ball_bridge': 1.9278497346033625,
 'lk_ball_bridge_uncpl': 12.636082331526154,
 'fa_elec': -371.0581733006344,
 'fa_intra_elec': -23.750217692233267,
 'pro_close': 0.0,
 'hbond_sr_bb': -131.23426068055278,
 'hbond_lr_bb': 0.0,
 'hbond_bb_sc': -2.4363384171592926,
 'hbond_sc': -43.67466710860555,
 'dslf_fa13': 0.0,
 'omega': 6.611153259821561,
 'fa_dun_dev': 8.992491434239561,
 'fa_dun_rot': 124.7017873669296,
 'fa_dun_semi': 160.12623193081222,
 'p_aa_pp': -39.03770718043659,
 'hxl_tors': 8.801517200407915,
 'ref': -94.55407000000004,
 'rama_prepro': -16.406240476720058,
 'gen_bonded': 0.0,
 'total_score': -470.6553446826023}

In [87]:
truncator.write_file("test_1.pdb", io.to_pdbstring(output_pose))

In [88]:
print("color deeppurple, all")
#print_sele(output_pose.scores, "hbnet_residues", "yellow")
print_sele(output_pose.scores, "core_bndr_polars", "orange")
print_sele(output_pose.scores, "core_SASA_polars", "red")


color deeppurple, all


KeyError: 'core_bndr_polars_pymol_selection'

In [ ]:
print("color deeppurple, all")
print_sele(output_pose.scores, "hbnet_residues", "blue")
print_sele(output_pose.scores, "core_hbnet_residues_chA", "yellow")
print_sele(output_pose.scores, "core_hbnet_residues_chB", "orange")
#print_sele(output_pose.scores, "core_SASA", "red")

In [ ]:

print("color deeppurple, all")
print_sele(output_pose.scores, "interface_by_vec", "orange")
print_sele(output_pose.scores, "core_bndr_and_interface_by_vec", "red")

In [20]:
print("color deeppurple, all")
print_sele(output_pose.scores, "interface_by_vec", "orange")
print_sele(output_pose.scores, "core_bndry_and_interface_by_vec", "red")

color deeppurple, all
select interface_by_vec, (chain A and resid 8,9,11,12,15,16,19,22,23,26,29,30,32,33,42,43,45,46,49,50,52,53,56,57,59,60,63,64,67) or (chain B and resid 88,89,91,92,95,96,98,99,102,103,105,106,109,112,113,120,123,124,127,130,131,134,135,137,138,141,142,144)
color orange, interface_by_vec


KeyError: 'core_bndry_and_interface_by_vec_pymol_selection'

NameError: name 'truncator' is not defined

In [1]:
output_pose.scores

NameError: name 'output_pose' is not defined

In [15]:
score_file = "test.sc" 
struct_name = "data/ZCON_37_N3_C1_2.pdb"
!/software/rosetta/latest/bin/rosetta_scripts -parser:protocol truncator/xml/80_test_selectors.xml -s {struct_name} \
    -out:file:scorefile  {score_file} \
    -print_pymol_selection 0 \
    -beta \
    -in:file:fullatom \
    -renumber_pdb 1 \
    -overwrite \
    -out:no_nstruct_label \
    -out:file:scorefile_format json > /dev/null
      

In [37]:
                          
scores = pd.read_json(score_file, lines=True).iloc[-1].to_dict()

In [47]:
print("color deeppurple, all")
print_sele(scores, "chainA", "orange")
print_sele(scores, "chainB", "red")

color deeppurple, all
select chainA, ( resid 92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119)
color orange, chainA
select chainB, ( resid 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91)
color red, chainB


In [19]:
print("color deeppurple, all")
print_sele(scores, "interface_by_vec", "orange")
print_sele(scores, "core_and_interface_by_vec", "red")

color deeppurple, all


NameError: name 'scores' is not defined